# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [2]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [3]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution

 solution = [None, False, False, True]


In [6]:

def ind_2_n(length, tup):
    out = 0
    for i in range(length):
        out = 2*out + tup[i]
            
    return out

def ind_2_col(l, a, g, b):
    L = pow(2,l)
    return ((ind_2_n(l, a))*L + ind_2_n(l, g))*2 + b 
        
# TODO: check correctness etc
def get_at(tup, a, l):
    # indexes from the left side of tup = (t0,0, t0,1, t1,0, t1,1, t2,0, t2,1)
    return tuple(tup[2*i + a[i]] for i in range(l))
 
def tuples_fixed_proj(l, proj_ind, proj_val):
#    h = randint(1,4000)
#    if (h == 5):
#        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
#        if (h == 5):
#            print 'modified x to ', x
        tuples.append(tuple(x[:]))
    return tuples    

# Assume a solution has been already found for client's variables
# Solve an artificial program for server variables just because it has
# nice support for accessing the server vars etc

def find_set_server_vars(l, solver, p0):
    
    print 'How much is server variable space limited by this solution & correctness?'
    serv_solver = MixedIntegerLinearProgram(solver='ppl')
    
    p1 = serv_solver.new_variable(integer = True, nonnegative=True)
    
    for i in range(3):                              
        for t in itertools.product(range(2), repeat = 2*l):
            serv_solver.add_constraint(p1[(i,t)] <= 1)
            
   # server's basic constraints
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l):
                for b in range(2):

                    if (solver.get_values(p0[(i,a,g,b)]) == 1):
                        all = tuples_fixed_proj(l, a, g)

                        for j in range(3):
                            if ((b == 0) and (i == j)) or ((b == 1) and (i != j)):
                                for t in all:
                                    serv_solver.add_constraint(p1[(j, t)] == 0)

    serv_solver.set_objective(sum(p1[(i,t)] for i in range(3) 
                                  for t in itertools.product(range(2), repeat = 2*l) ))
    
    res = serv_solver.solve()
    print '# of solver vars remaining = ', res


# Hack from correctness and server security (!) 
# strength = 0 is consistent with standard LP. strength = 1 attempts to better sort things out   

def add_no_id_a(solver, p0, q1, strength = 0):
    
    if strength == -1:
        return
    
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for j in range(3):
                if (j > i):
                    # this is relaxed
                    if strength == 0:
                        solver.add_constraint(sum(q1[(i, a, zeros, b)] for b in range(2)) + sum(q1[(j, a, zeros, b)] 
                                                 for b in range(2)) <= 1)    
                    elif strength == 1:
                        for b1 in range(2):
                            for b2 in range(2):
                                solver.add_constraint(p0[(i, a, zeros, b1)] + p0[(i, a, zeros, b2)] <= 1)
                                
# TODO: change the name to something more informative.
# Hack #2 from correctness: For every i,a,b with p^i_a > 0, there exists g so that (i,a,g,b) > 0.
# This requires some trial and error. Otherwise, one xi at least remains without TO-input options
# This will eventually not be part of the client constraints. Currently used to `direct' the soutions.
    
def add_a_balance(solver, p0, q1, strength = 0):
    if strength == -1:
        return
    
    if strength == 1:    
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - 1)
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      >= 1 - pow(2, l))

    # Much weaker, but no integers..
    elif strength == 0:
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(q1[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - my_eps)
                solver.add_constraint(sum(q1[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l))
                                      <= pow(2, l) - my_eps)


# Another hack from correctness. For all b, there exists at least one value of t, so 
# that every a reading this g will output b. Thus, the resulting sum of probabilities for
# that g is 1. There may be additional g's (the g results from some V assigned to server for each output
# value).                
def add_b_balance(solver, q1, strength = 0):
    if strength == -1:
        return 
    
    if strength == 0:
        for i in range(3):                              
            for b in range(2):
                # this is a relaxation - allows for splitting among several g's
                solver.add_constraint(sum(q1[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                      for g in itertools.product(range(2), repeat = l)) >= 1)                

# only captures a small subset of constraints                
def test_linear_solutions(l):
    
        b = []
        constraints = []
        
        print 'generating constraints ...'
        # needs strengthening into 3 equations
        cur_row = [0 for i in range(2*pow(2, 2*l))]
        for a in itertools.product(range(2), repeat = l):
            cur_row[ind_2_col(l, a, zeros, 0)] = 1
            cur_row[ind_2_col(l, a, zeros, 1)] = 1
        constraints.append(cur_row)
        b.append(3)
        
        
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l): 
                cur_row = [0 for i in range(2*pow(2, 2*l))]
                
                cur_row[ind_2_col(l,a,zeros,0)] = 1
                cur_row[ind_2_col(l,a,zeros,1)] = 1
                cur_row[ind_2_col(l,a,g,0)] = -1
                cur_row[ind_2_col(l,a,g,1)] = -1
                b.append(0)
                constraints.append(cur_row)

                    
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(2*pow(2, 2*l))]
            b.append(1)
            for a in itertools.product(range(2), repeat = l):
                cur_row[ind_2_col(l, a, get_at(t,a,l), 1)] = 1
            constraints.append(cur_row)
            
        
        m_constr = matrix(QQ,constraints)
        right_side = vector(QQ,b)
        print 'Created linear system of rank',m_constr.rank(),'/',pow(2, 2*l + 1)
        try:
            v = m_constr\right_side
            ker = m_constr.right_kernel()
            print 'solution = ',v
            print '=================================================='
            print 'Kernel = ', ker
        except:
            print 'There is no solution!'
                
                
# main     

# global variables

my_eps = QQ(1/10000)

    
for l in range(2,7):
    import time
    t0 = time.time()
    print 'finding a solution for l=',l
    zeros = tuple((0 for i in range(l)))
    
    try:
        test_linear_solutions(l)
        
        # find_solution(l, 'client', 400 - i)
    finally:
        print 'time =', time.time() - t0,'\n'


finding a solution for l= 2
generating constraints ...
Created linear system of rank 25 / 32
There is no solution!
time = 0.0048828125 

finding a solution for l= 3
generating constraints ...
Created linear system of rank 91 / 128
There is no solution!
time = 0.0421781539917 

finding a solution for l= 4
generating constraints ...
Created linear system of rank 337 / 512
There is no solution!
time = 1.01528096199 

finding a solution for l= 5
generating constraints ...
Created linear system of rank 1267 / 2048
There is no solution!
time = 134.827276945 

finding a solution for l= 6
generating constraints ...
Created linear system of rank 4825 / 8192
There is no solution!
time = 29930.3087771 

